In [5]:
import tensorflow as tf
import joblib, pathlib
import numpy as np


# Añadir src/ al path para poder importar config
PROJECT_ROOT = pathlib.Path().resolve().parent.parent  
if str(PROJECT_ROOT) not in sys.path:
    sys.path.insert(0, str(PROJECT_ROOT))
from src import config as cfg

# --- Cargar dataset procesado ---
ruta = cfg.DATA / "processed" / "gru5d_data.pkl"
print("📦 Cargando datos desde:", ruta)

data = joblib.load(ruta)

X, y = data["X"], data["y"]
fechas = data["dates"]
print(f"✅ X shape: {X.shape}")
print(f"✅ y shape: {y.shape}")
print(f"📅 Fechas disponibles: {fechas.min()} → {fechas.max()}")

📦 Cargando datos desde: C:\Users\ferra\Documents\TFM\data\processed\gru5d_data.pkl
✅ X shape: (4450, 60, 80)
✅ y shape: (4450, 40)
📅 Fechas disponibles: 2012-08-22 00:00:00 → 2025-06-21 00:00:00


In [16]:
# --- División temporal ---
train_mask = fechas < "2019-01-01"
val_mask   = (fechas >= "2019-01-01") & (fechas < "2021-01-01")
test_mask  = fechas >= "2021-01-01"

X_train, y_train = X[train_mask], y[train_mask]
X_val, y_val     = X[val_mask], y[val_mask]
X_test, y_test   = X[test_mask], y[test_mask]

print(f"🔹 Train: {X_train.shape}")
print(f"🔹 Val:   {X_val.shape}")
print(f"🔹 Test:  {X_test.shape}")

print("🗓️ Rango fechas:")
print("Train:", fechas[train_mask].min(), "→", fechas[train_mask].max())
print("Val:  ", fechas[val_mask].min(), "→", fechas[val_mask].max())
print("Test: ", fechas[test_mask].min(), "→", fechas[test_mask].max())

🔹 Train: (2086, 60, 80)
🔹 Val:   (731, 60, 80)
🔹 Test:  (1633, 60, 80)
🗓️ Rango fechas:
Train: 2012-08-22 00:00:00 → 2018-12-31 00:00:00
Val:   2019-01-01 00:00:00 → 2020-12-31 00:00:00
Test:  2021-01-01 00:00:00 → 2025-06-21 00:00:00


In [18]:
# --- Crear modelo GRU ---
print("🛠️  Definiendo arquitectura...")

inputs = tf.keras.Input(shape=X.shape[1:])
x = tf.keras.layers.GRU(64, return_sequences=True)(inputs)
x = tf.keras.layers.GRU(32)(x)
outputs = tf.keras.layers.Dense(y.shape[1])(x)

model = tf.keras.Model(inputs=inputs, outputs=outputs)

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
    loss="mse",
    metrics=[tf.keras.metrics.RootMeanSquaredError()]
)

model.summary()


🛠️  Definiendo arquitectura...


Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)      │ (None, 60, 80)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ gru_2 (GRU)                     │ (None, 60, 64)         │        28,032 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ gru_3 (GRU)                     │ (None, 32)             │         9,408 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 40)             │         1,320 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 38,760 (151.41 KB)

 Trainable params: 38,760 (151.41 KB)

 Non-trainable params: 0 (0.00 B)

In [13]:
# --- Entrenamiento ---
ckpt_path = pathlib.Path(cfg.MODELS) / "gru5d.keras"
print("📁 Checkpoint se guardará en:", ckpt_path)


print("🔍 NaNs en X:", np.isnan(X).sum())
print("🔍 NaNs en y:", np.isnan(y).sum())
print("🔍 Inf en X :", np.isinf(X).sum())
print("🔍 Inf en y :", np.isinf(y).sum())
print("🔍 Rango X  :", np.min(X), "→", np.max(X))
print("🔍 Rango y  :", np.min(y), "→", np.max(y))

es_cb = tf.keras.callbacks.EarlyStopping(patience=5, restore_best_weights=True)

history = model.fit(
    X, y,
    validation_split=0.2,
    epochs=50,
    batch_size=32,
    callbacks=[es_cb],
    verbose=2
)


print("🏁 Entrenamiento finalizado.")

📁 Checkpoint se guardará en: C:\Users\ferra\Documents\TFM\models\gru5d.keras
🔍 NaNs en X: 0
🔍 NaNs en y: 0
🔍 Inf en X : 0
🔍 Inf en y : 0
🔍 Rango X  : -6.9616404 → 7.819829
🔍 Rango y  : -0.15001197 → 1.2250828
Epoch 1/50
112/112 - 14s - 121ms/step - loss: 0.0045 - root_mean_squared_error: 0.0671 - val_loss: 8.7161e-04 - val_root_mean_squared_error: 0.0295
Epoch 2/50
112/112 - 8s - 76ms/step - loss: 5.0760e-04 - root_mean_squared_error: 0.0225 - val_loss: 3.4040e-04 - val_root_mean_squared_error: 0.0185
Epoch 3/50
112/112 - 8s - 73ms/step - loss: 2.9298e-04 - root_mean_squared_error: 0.0171 - val_loss: 2.0865e-04 - val_root_mean_squared_error: 0.0144
Epoch 4/50
112/112 - 9s - 76ms/step - loss: 2.2710e-04 - root_mean_squared_error: 0.0151 - val_loss: 1.6494e-04 - val_root_mean_squared_error: 0.0128
Epoch 5/50
112/112 - 9s - 82ms/step - loss: 1.9882e-04 - root_mean_squared_error: 0.0141 - val_loss: 1.3938e-04 - val_root_mean_squared_error: 0.0118
Epoch 6/50
112/112 - 10s - 85ms/step - loss

In [20]:
print("📊 Evaluando en test...")
y_pred = model.predict(X_test)
rmse = np.sqrt(((y_test - y_pred)**2).mean(axis=0))
rmse_mean = rmse.mean()
print("✅ RMSE medio:", rmse_mean)

# --- Guardado ---
model.save(cfg.MODELS / "gru5d.keras")
joblib.dump(history.history, cfg.RESULT / "history_gru5d.pkl")
joblib.dump(rmse_mean, cfg.RESULT / "rmse_gru5d.pkl")

print("✅ Modelo y resultados guardados correctamente.")

📊 Evaluando en test...
52/52 ━━━━━━━━━━━━━━━━━━━━ 1s 21ms/step
✅ RMSE medio: 0.17298225
✅ Modelo y resultados guardados correctamente.
